In [2]:
from ktms import *
from MAInit import *

In [3]:
#############################################################################3
# name = "CoPt3_100_u2"
# name = 'Pt_100_u2_no20_CO_n1_d9'
name = 'Pt_100_u2'
# name = 'Pt_100_u2_no20_CO_n1_d9'
# name = 'Cu_100_u2_no20_CO_n1_d8'

inname = name + '.traj'
atoms = init_query(inname, 'spacom')
bareatoms, poslis = removemolecule(atoms, ['C', 'O'])

In [4]:
barestruct = AseAtomsAdaptor.get_structure(bareatoms)
baresites = getadsites(bareatoms, False)
d = 1.13

maxmole = 5
mindist = 1.8 # must be not 0
molecule = Atoms('CO', positions=[(0., 0., 0.), (0., 0., d)], cell=(10., 10., 10.))

In [5]:
sites0_ = baresites['all']
sites0 = [list(i) for i in sites0_]

group = creategroup(bareatoms, sites0)

rused = []
for i in reversed(range(len(sites0))):
    for j in range(len(poslis)):
        if np.allclose(sites0[i][:2], poslis[j]):
            tmp = sites0.pop(i)
            rused.append(tmp)
baresites, rused

Symmetrically same structure found!
0 same config found!
Symmetrically same structure found!
0 same config found!
Symmetrically same structure found!
0 same config found!
Symmetrically same structure found!
1 same config found!
Symmetrically same structure found!
1 same config found!
Symmetrically same structure found!
1 same config found!
Symmetrically same structure found!
1 same config found!
Symmetrically same structure found!
2 same config found!
Symmetrically same structure found!
1 same config found!
Symmetrically same structure found!
1 same config found!
Symmetrically same structure found!
2 same config found!
Symmetrically same structure found!
1 same config found!
Symmetrically same structure found!
2 same config found!


({'ontop': [array([ 0.       ,  0.       , 17.9862008]),
   array([ 2.82192212,  0.        , 17.9862008 ]),
   array([ 0.        ,  2.82192212, 17.9862008 ]),
   array([ 2.82192212,  2.82192212, 17.9862008 ])],
  'bridge': [array([ 5.64384424,  4.23288318, 17.9862008 ]),
   array([ 2.82192212,  1.41096106, 17.9862008 ]),
   array([ 1.41096106,  2.82192212, 17.9862008 ]),
   array([ 1.41096106,  1.41096106, 17.9862008 ]),
   array([ 0.        ,  1.41096106, 17.9862008 ]),
   array([ 4.23288318,  1.41096106, 17.9862008 ]),
   array([ 4.23288318,  2.82192212, 17.9862008 ]),
   array([ 4.23288318,  4.23288318, 17.9862008 ]),
   array([ 2.82192212,  4.23288318, 17.9862008 ]),
   array([ 4.23288318,  5.64384424, 17.9862008 ]),
   array([ 1.41096106,  4.23288318, 17.9862008 ]),
   array([ 1.41096106,  5.64384424, 17.9862008 ])],
  'hollow': [],
  'all': [array([ 0.       ,  0.       , 17.9862008]),
   array([ 2.82192212,  0.        , 17.9862008 ]),
   array([ 0.        ,  2.82192212, 17.98620

In [6]:
group

[[[0.0, 0.0, 17.986200796545],
  [2.8219221178541876, 0.0, 17.986200796545],
  [0.0, 2.8219221178541876, 17.986200796545],
  [2.8219221178541876, 2.8219221178541876, 17.986200796545]],
 [[2.8219221178541876, 4.232883176781282, 17.986200796545],
  [1.410961058927089, 0.0, 17.986200796545],
  [5.643844235708371, 4.232883176781282, 17.986200796545],
  [1.410961058927089, 2.8219221178541853, 17.986200796545],
  [2.8219221178541853, 1.4109610589270927, 17.986200796545],
  [0.0, 1.4109610589270927, 17.986200796545],
  [4.232883176781282, 5.643844235708375, 17.986200796545],
  [4.232883176781282, 2.821922117854188, 17.986200796545]],
 [[1.410961058927089, 4.232883176781282, 17.986200796545],
  [1.410961058927089, 1.4109610589270927, 17.986200796545],
  [4.232883176781282, 1.4109610589270927, 17.986200796545],
  [4.232883176781282, 4.2328831767812805, 17.986200796545]]]

In [7]:
allatoms, allused, mindistlis, numdict, molenum \
    = getuniqueatoms(atoms, bareatoms, sites0, maxmole, mindist, rused, group, molecule)

Used initialized!
1-------Pt16CO-------
2-------Pt16COCO-------
3-------Pt16COCOCO-------
4-------Pt16COCOCOCO-------
Distance 1.41 is below 1.8
5-------Pt16COCOCOCOCO-------
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
4-------Pt16COCOCOCO-------
Symmetrically same structure found!
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
5-------Pt16COCOCOCOCO-------
Distance 1.41 is below 1.8
5-------Pt16COCOCOCOCO-------
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
3-------Pt16COCOCO-------
Symmetrically same structure found!
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
4-------Pt16COCOCOCO-------
Symmetrically same structure found!
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
Distance 1.41 is below 1.8
5-------Pt16COCOCOCOCO-------
5-------Pt16COCOCOCOCO-------
Distance 1.41 is below 1.8
4-------Pt16COCOCOCO-------
Symmetrically same structure

Symmetrically same structure found!
Used initialized!
1-------Pt16CO-------
2-------Pt16COCO-------
3-------Pt16COCOCO-------
4-------Pt16COCOCOCO-------
2-------Pt16COCO-------
Symmetrically same structure found!


In [8]:
print(len(allatoms))

for num in numdict:
    print(num, numdict[num])

68
1 3
2 11
3 17
4 24
5 13


In [9]:
mindistlis

[5.643844235708376,
 2.8219221178541876,
 2.8219221178541876,
 2.8219221178541876,
 1.9954002655149954,
 1.9954002655149954,
 1.9954002655149952,
 1.9954002655149954,
 1.995400265514996,
 1.9954002655149952,
 1.9954002655149952,
 1.9954002655149952,
 1.995400265514996,
 1.995400265514996,
 1.995400265514996,
 2.8219221178541876,
 2.8219221178541822,
 3.9908005310299997,
 1.995400265514996,
 1.9954002655149952,
 1.9954002655149952,
 1.995400265514996,
 3.1550048413660683,
 1.9954002655150052,
 1.9954002655150003,
 1.9954002655149985,
 1.995400265514999,
 2.8219221178541867,
 1.9954002655149952,
 1.9954002655149952,
 1.995400265514996,
 1.9954002655149952,
 1.9954002655149952,
 1.9954002655149952,
 1.995400265514996,
 5.643844235708375,
 1.9954002655150032,
 1.9954002655149998,
 1.9954002655149998,
 1.9954002655149998,
 1.9954002655149998,
 1.9954002655149998,
 1.9954002655149952,
 1.9954002655149954,
 1.9954002655149954,
 1.9954002655149954,
 1.9954002655149998,
 1.9954002655149954,
 1.

In [10]:
count = 0
indexlist = []
for i in range(len(mindistlis)):
    if mindistlis[i] > 2.5:
        indexlist.append(i)
        count += 1
# count, indexlist

In [11]:
for i in range(len(allatoms)): # len(allatoms)
    if 'COCOCOCOCO' in str(allatoms[i].symbols):
        view(allatoms[i])
        print(allatoms[i].symbols)
#         print('a')

Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO
Pt16COCOCOCOCO


In [30]:
for i in range(len(allatoms)):
    if 'COCOCOCOCO' in str(allatoms[i].symbols):
        outname = name + str('_no') + str('{0:03d}'.format(i+1)) +'_CO_n' + str(molenum[i]) + str('_d') + str(int(np.ceil(mindistlis[i]/0.5)-3))  + '.traj'
        print(outname)
        outpath = '/home/katsuyut/init/' + str(outname)
#         allatoms[i].write(outpath)

Pt_100_u2_no005_CO_n5_d1.traj
Pt_100_u2_no007_CO_n5_d1.traj
Pt_100_u2_no008_CO_n5_d1.traj
Pt_100_u2_no011_CO_n5_d1.traj
Pt_100_u2_no012_CO_n5_d1.traj
Pt_100_u2_no021_CO_n5_d1.traj
Pt_100_u2_no026_CO_n5_d1.traj
Pt_100_u2_no034_CO_n5_d1.traj
Pt_100_u2_no040_CO_n5_d1.traj
Pt_100_u2_no041_CO_n5_d1.traj
Pt_100_u2_no043_CO_n5_d1.traj
Pt_100_u2_no044_CO_n5_d1.traj
Pt_100_u2_no045_CO_n5_d1.traj
